# 2. Preprocesado de texto

### Importar librerías necesarias

Cargar los archivos previamente en el entorno de Google Colab.

In [ ]:
!pip install -r requirements.txt

  Using cached contractions-0.1.73-py2.py3-none-any.whl (8.7 kB)
  Using cached jellyfish-1.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached negspacy-1.0.4.tar.gz (13 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached num2words-0.5.13-py3-none-any.whl (143 kB)
  Using cached plotly_express-0.4.1-py2.py3-none-any.whl (2.9 kB)
  Using cached pyDAWG-1.0.1.tar.gz (28 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of pyldavis to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from utils import *
from preprocessing import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


### Cargar Datos desde Archivo CSV

Debemos subir previamente el archivo al entorno de Google Colab o cargarlo desde Google Drive.

#### Google Drive

In [ ]:
# df = load_data_drive('Colab Notebooks/Sports_and_Outdoors/sample_data.csv')

Google Drive ya está montado.
Archivo cargado exitosamente desde: /content/drive/My Drive/Colab Notebooks/Sports_and_Outdoors/sample_data.csv


#### Archivos locales

In [ ]:
!gunzip sample_data_balanced_complete.csv.gz  # Descomprimimos el archivo

In [ ]:
df = load_data('sample_data_balanced_complete.csv')

In [ ]:
print(df.columns)

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'sentiment',
       'cleaned_text', 'cleaned_text_exclude_numbers', 'text_length'],
      dtype='object')


In [ ]:
print(df['text'].head()) # Ver resultados

0    The travel mug was OK, not bad but the coffee ...
1    I'm terrible with planks so I was terrible at ...
2    Drawstring immediately ripped inches of stitch...
3    The black face is cracking and falling off aft...
4           Leaves white stuff in your hair. Not good!
Name: text, dtype: object


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rating                        50000 non-null  float64
 1   title                         49999 non-null  object 
 2   text                          49998 non-null  object 
 3   images                        50000 non-null  object 
 4   asin                          50000 non-null  object 
 5   parent_asin                   50000 non-null  object 
 6   user_id                       50000 non-null  object 
 7   timestamp                     50000 non-null  int64  
 8   helpful_vote                  50000 non-null  int64  
 9   verified_purchase             50000 non-null  bool   
 10  sentiment                     50000 non-null  object 
 11  cleaned_text                  49920 non-null  object 
 12  cleaned_text_exclude_numbers  49918 non-null  object 
 13  t

In [ ]:
# Eliminamos valores NaN
df['text'] = df['text'].fillna('')
df['title'] = df['title'].fillna('')

Aunque se realizó un pequeño preprocesamiento en los datos en la fase anterior, no se tomará en cuenta. A partir de ahora, se trabajará en la columna `'text'` desde cero (etapa de preprocesamiento).

# Preprocesado

## Pipeline con SpaCy

In [ ]:
# Seleccionamos una muestra de reseñas para el análisis
sample_reviews = df['text'].sample(n=40)  # Podemos ajustar 'n' al número de reseñas que queremos explorar

# Función para identificar y mostrar los tipos de entidades en las reseñas
def identify_entity_types(reviews):
    entity_types = set()

    for review in reviews:
        doc = nlp(review)
        for ent in doc.ents:
            entity_types.add((ent.text, ent.label_))

    return entity_types

# Identificamos los tipos de entidades en la muestra de reseñas
entity_types_in_sample = identify_entity_types(sample_reviews)

# Imprimimos los tipos de entidades encontrados
for text, label in entity_types_in_sample:
    print(f"Texto: {text}, Tipo de Entidad: {label}")

Texto: 168mm, Tipo de Entidad: QUANTITY
Texto: first, Tipo de Entidad: ORDINAL
Texto: 29, Tipo de Entidad: CARDINAL
Texto: half, Tipo de Entidad: CARDINAL
Texto: BOTH, Tipo de Entidad: NORP
Texto: these days, Tipo de Entidad: DATE
Texto: at least 4 weeks, Tipo de Entidad: DATE
Texto: Time, Tipo de Entidad: ORG
Texto: month, Tipo de Entidad: DATE
Texto: the 90 day, Tipo de Entidad: DATE
Texto: today, Tipo de Entidad: DATE
Texto: 25%, Tipo de Entidad: PERCENT
Texto: daily, Tipo de Entidad: DATE
Texto: winter, Tipo de Entidad: DATE
Texto: 550, Tipo de Entidad: CARDINAL
Texto: 8 years, Tipo de Entidad: DATE
Texto: 6 inches, Tipo de Entidad: QUANTITY
Texto: Iron Bull Strength, Tipo de Entidad: ORG
Texto: the Fat Grips Pro, Tipo de Entidad: EVENT
Texto: Alpha Grips, Tipo de Entidad: PERSON
Texto: 10 or 15 minutes, Tipo de Entidad: TIME
Texto: 1, Tipo de Entidad: CARDINAL
Texto: 4, Tipo de Entidad: CARDINAL
Texto: the summer, Tipo de Entidad: DATE
Texto: Great Customer Service, Tipo de Entida

### Pipeline 1: Preprocesamiento con spaCy

In [ ]:
# Ejemplo de uso de la función (prueba)
sample_review = "I'm not loving the new tent I bought. It isn't extremely good!"
cleaned_review = clean_text_spacy(sample_review)
print(cleaned_review)

love new tent buy extremely good


En este primer Pipeline, empleamos spaCy y NegSpacy para el preprocesamiento del texto, con el fin de mejorar la detección de negaciones en el contexto de entidades nombradas. A pesar de que este enfoque enriquece el análisis, su limitación reside en no modificar directamente el texto para reflejar las negaciones. Esta consideración nos lleva a plantear otros pipelines.

## Pipelines con NLTK

### Pipeline 2: Limpieza de Texto Básica con NLTK

In [ ]:
print(clean_text("I'm not loving the new tent I bought. It's extremely good!")) # Ejemplo de uso de la función (prueba)

love new tent bought extrem good


In [ ]:
# Uso de la función de limpieza en el DataFrame
df['cleaned_text'] = df['text'].apply(clean_text)

In [ ]:
print(df['text'].head(20))

0     The travel mug was OK, not bad but the coffee ...
1     I'm terrible with planks so I was terrible at ...
2     Drawstring immediately ripped inches of stitch...
3     The black face is cracking and falling off aft...
4            Leaves white stuff in your hair. Not good!
5     Too large and not for little girls. More for a...
6     Update... after charging approximately 3 times...
7     you may be fooled by soft inner padding, but t...
8     I purchased two of these.  I put them on my do...
9     Missing hardware and pieces didn't fit togethe...
10    Look, this is probably my fault, but I thought...
11    I wanted to love this... a few of us bought th...
12    just so not what i was thinking it was gonna b...
13    Bought this as a small gift for a Scouting Fri...
14    The water bottle silicone coat smells strongly...
15                                     Not true to size
16    This is advertised as one full quart, but by w...
17    I am posting a "reserved" review...After t

In [ ]:
print(df['cleaned_text'].head(20))

0     travel mug wa ok bad coffe mug team emblem wa ...
1     terribl plank wa terribl thi mayb ab need lot ...
2           drawstr immedi rip inch stitch produci unus
3            black face crack fall two day poor qualiti
4                            leaf white stuff hair good
5                      larg littl girl adult older teen
6     updat charg approxim three time turn anyth cho...
7     may fool soft inner pad skate terribl boot sid...
8     purchas two put dog cute later notic fall apar...
9     miss hardwar piec fit togeth like thi wa garba...
10    look thi probabl fault thought purchas bear ba...
11    want love thi u bought tv show work love first...
12                   wa think wa go much smaller appear
13    bought thi small gift scout friend like tanto ...
14    water bottl silicon coat smell strongli like p...
15                                            true size
16    thi advertis one full quart weight onli weigh ...
17    post reserv reviewaft experi buy black one

### Pipeline 3: Limpieza de Texto para Modelos Contextuales

In [ ]:
print(clean_text_for_contextual_models("I bought 2 of these for my hiking trip for $5.99 each. They're amazing!")) # Ejemplo de uso (prueba)

bought two hiking trip five hundred and ninety-nine amazing


### Pipeline 4: Función de limpieza que excluye los números

In [ ]:
print(clean_text_exclude_numbers("I bought 2 of these for my hiking trip for $5.99 each. They're amazing!")) # Ejemplo de uso (prueba)

bought hiking trip amazing


In [ ]:
# Reemplazar las columnas existentes con los nuevos textos limpios
df['cleaned_text'] = df['text'].apply(clean_text_for_contextual_models)
df['cleaned_text_exclude_numbers'] = df['text'].apply(clean_text_exclude_numbers)

In [ ]:
print(df[['text','cleaned_text', 'cleaned_text_exclude_numbers']].head(10))

                                                text  \
0  The travel mug was OK, not bad but the coffee ...   
1  I'm terrible with planks so I was terrible at ...   
2  Drawstring immediately ripped inches of stitch...   
3  The black face is cracking and falling off aft...   
4         Leaves white stuff in your hair. Not good!   
5  Too large and not for little girls. More for a...   
6  Update... after charging approximately 3 times...   
7  you may be fooled by soft inner padding, but t...   
8  I purchased two of these.  I put them on my do...   
9  Missing hardware and pieces didn't fit togethe...   

                                        cleaned_text  \
0  travel mug ok bad coffee mug team emblem affix...   
1  terrible planks terrible maybe abs need lot st...   
2  drawstring immediately ripped inches stitches ...   
3  black face cracking falling two days poor quality   
4                       leaves white stuff hair good   
5              large little girls adults older 

In [39]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rating                        50000 non-null  float64
 1   title                         50000 non-null  object 
 2   text                          50000 non-null  object 
 3   images                        50000 non-null  object 
 4   asin                          50000 non-null  object 
 5   parent_asin                   50000 non-null  object 
 6   user_id                       50000 non-null  object 
 7   timestamp                     50000 non-null  int64  
 8   helpful_vote                  50000 non-null  int64  
 9   verified_purchase             50000 non-null  bool   
 10  sentiment                     50000 non-null  object 
 11  cleaned_text                  50000 non-null  object 
 12  cleaned_text_exclude_numbers  50000 non-null  object 
 13  t

### Guardar el DataFrame Modificado

In [ ]:
# save_data_drive(df,'processed_data.csv')  # En Google Drive

DataFrame guardado en: /content/drive/My Drive/Colab Notebooks/Sports_and_Outdoors/cleaned_reviews.csv


In [40]:
def save_data_local(df, filename):
    """
    Guarda un DataFrame en un archivo CSV localmente.

    Parámetros:
    - df: DataFrame a guardar.
    - filename: Nombre del archivo para guardar el DataFrame.
    """
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"DataFrame guardado localmente como: {filename}")

In [41]:
# Guardar como CSV
save_data_local(df,'processed_data.csv')

DataFrame guardado localmente como: processed_data.csv


In [37]:
import gzip
import shutil

with open('processed_data.csv', 'rb') as f_in:
    with gzip.open('processed_data.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

f_out.close()
f_in.close()

Durante el proceso de preprocesamiento y almacenamiento de los datos en un archivo CSV, se encontraron valores nulos inesperados al recargar el DataFrame en el archivo `3-entrenamiento-y-testeo-modelo`. Este problema no ocurrió al utilizar el formato Parquet para guardar y cargar los datos.

La diferencia en el comportamiento se debe a cómo cada formato maneja la serialización y deserialización de los datos. **CSV** es un formato de texto simple que puede interpretar incorrectamente ciertos caracteres especiales o estructuras de datos complejas, lo que lleva a la pérdida o corrupción de información. **Parquet**, por otro lado, es un formato binario columnar optimizado para datos tabulares, que preserva de manera más efectiva los tipos de datos y la integridad de la información, evitando así los problemas observados con el formato CSV.

Este hallazgo subraya la importancia de elegir el formato adecuado para el almacenamiento de datos, especialmente cuando se trabaja con texto y estructuras de datos complejas.

In [45]:
data_csv = pd.read_csv('processed_data.csv', encoding='utf-8'). # Cargar CSV

# Verificar nuevamente valores nulos
print(data_csv.isnull().sum())

rating                           0
title                            1
text                             2
images                           0
asin                             0
parent_asin                      0
user_id                          0
timestamp                        0
helpful_vote                     0
verified_purchase                0
sentiment                        0
cleaned_text                    80
cleaned_text_exclude_numbers    82
text_length                      0
dtype: int64


In [44]:
# Guardar como Parquet
df.to_parquet('processed_data.parquet')

# Cargar Parquet
data_parquet = pd.read_parquet('processed_data.parquet')

# Verificar nuevamente valores nulos
print(data_parquet.isnull().sum())

rating                          0
title                           0
text                            0
images                          0
asin                            0
parent_asin                     0
user_id                         0
timestamp                       0
helpful_vote                    0
verified_purchase               0
sentiment                       0
cleaned_text                    0
cleaned_text_exclude_numbers    0
text_length                     0
dtype: int64


## Pipeline de procesamiento

En este cuaderno Jupyter, se han explorado y desarrollado diversas técnicas de preprocesamiento de texto adaptadas a las necesidades específicas del proyecto. A lo largo del proceso, se han ajustado y optimizado las funciones de limpieza.

Para facilitar la **reutilización y mejorar la organización del código**, se ha consolidado este trabajo en un **pipeline de preprocesamiento** bien documentado, ubicado en el archivo `preprocessing.py`. Este archivo contiene todas las funciones de preprocesamiento definidas de manera modular, lo que permite una integración sencilla y directa en futuros proyectos o etapas de análisis dentro de este mismo proyecto.

# Conclusiones

1. **Filtrado Efectivo**: La estrategia de eliminar stopwords y palabras de bajo valor semántico se añade con el objetivo de enfocar el análisis en el contenido más relevante. Sin embargo, es importante recalcar que la selección de estas palabras debe ser revisada continuamente para asegurar su pertinencia al contexto específico del análisis.

2. **Uso de Beautiful Soup**: La decisión de integrar Beautiful Soup fue una respuesta directa a la presencia de elementos HTML no deseados y repetidos "br" identificados durante la fase exploratoria. A pesar de que esta inclusión puede impactar ligeramente la eficiencia, los beneficios en términos de la calidad del texto justifican su uso.

3. **Manejo de Números con `num2words`**: La adopción de `num2words` enriquece el análisis al transformar los dígitos en texto, lo que facilita la exploración de patrones asociados a cantidades, precios o características numéricas en las reseñas.

4. **Preservación del Contexto para Modelos Contextuales**: La decisión de minimizar las alteraciones agresivas en las formas de las palabras apunta a retener tanto el contexto como la semántica del texto.

5. **Personalización y Flexibilidad**: Este enfoque proporciona una sólida base inicial para el preprocesamiento de texto. No obstante, es fundamental entenderlo como un punto de partida flexible, susceptible de ser adaptado o modificado en función de los requerimientos específicos del análisis o del conjunto de datos.

6. **Balance entre Eficiencia y Efectividad**: Mientras que las primeras estrategias de preprocesamiento priorizaban la eficiencia al consolidar el vocabulario, la aproximación final destaca por su capacidad para capturar la complejidad semántica del texto, importante para análisis basados en el contexto.

### Mejoras Futuras

A pesar de que el enfoque actual se centra en la manipulación individual de palabras, es crucial reconocer las limitaciones inherentes a esta metodología, especialmente en lo que respecta a la captura de negaciones y expresiones idiomáticas. Futuras iteraciones podrían beneficiarse de incorporar técnicas que permitan un análisis más profundo del contexto y las relaciones entre palabras, posiblemente a través del uso de modelos de lenguaje avanzados.

Además, es recomendable una evaluación continua y ajuste de los criterios de filtrado y limpieza para alinearlos estrechamente con las metas del proyecto y los hallazgos emergentes del análisis exploratorio. Este enfoque iterativo y reflexivo no solo mejora la precisión del análisis, sino que también garantiza la relevancia y actualidad del proceso de preprocesamiento.